In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 30.

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
import numpy as np
import os

In [7]:
def text_embedding(text) -> None:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text) # Define a function to create text embeddings using the SentenceTransformer model.

In [8]:
def generate_context(query):
    vector=text_embedding(query).tolist()

    results=collection.query(
        query_embeddings=vector,
        n_results=15,
        include=["documents"]
    )

    res = "\n".join(str(item) for item in results['documents'][0])
    return res  # Define a function that generates context by finding documents similar to a query embedding.

In [9]:
def chat_completion(system_prompt, user_prompt,length=1000):
    final_prompt=f"""<s>[INST]<<SYS>>
    {system_prompt}
    <</SYS>>

    {user_prompt} [/INST]"""
    return client.text_generation(prompt=final_prompt,max_new_tokens = length).strip() # Define a function to generate chat responses using the context provided.

In [10]:
!pwd

/content


In [12]:
df=pd.read_csv('/content/oscars.csv') # Read the Oscars data for the year 2023 into a DataFrame, preprocess the data, and create context sentences.
df=df.loc[df['year_ceremony'] == 2023]
df=df.dropna(subset=['film'])
df.loc[:, 'category'] = df['category'].str.lower()
df.loc[:, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' to win the award'
df.loc[df['winner'] == False, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' but did not win'

In [13]:
client = chromadb.Client() # Instantiate a ChromaDB client and create or get an existing collection for the Oscars 2023.
collection = client.get_or_create_collection("oscars-2023")

In [14]:
docs=df["text"].tolist()
ids= [str(x) for x in df.index.tolist()]
collection.add(
    documents=docs,
    ids=ids
) # Prepare the documents and their IDs to be added to the ChromaDB collection

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 74.8MiB/s]


In [15]:
!pip install transformers

URI='http://139.84.142.100:8080'
client = InferenceClient(model=URI) # Initialize an inference client pointing to the specified URI.

In [16]:
#query="What did Ke Huy Quan work on?"
#query="Which movie won the best music award?"
query="Did Lady Gaga win an award at Oscars 2023?"
#query="Who is the music director of RRR?"
context=generate_context(query) #generate context from the query

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
context

'Monika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howard Gertler, John Lyons, Na

In [18]:
system_prompt="""\
You are a helpful AI assistant that can answer questions on Oscar 2023 awards. Answer based on the context provided. If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
""" # Define a system prompt for guiding the AI in generating responses.

In [19]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
""" # Format the user prompt with the context and query.

In [20]:
user_prompt # Print the user prompt for verification.

'\nBased on the context:\nMonika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howar

In [21]:
!pip install openai
!pip install huggingface_hub # Import the openai package and utilities from the Hugging Face Hub.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00


In [22]:
import openai
from huggingface_hub import HfFolder

In [32]:
from transformers import pipeline

# For LLaMA 2, assuming it is available directly from Hugging Face and is public
model_name = "meta-llama/Llama-2-7b"

In [ ]:
chat_completion(system_prompt,user_prompt) # Generate a chat completion response using the system prompt and user prompt, the output indicates that Lady Gaga's nomination status at the Oscars 2023.

Lady Gaga was nominated but did not win at the Oscars 2023.